In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [ ]:
# Create a reference the CSV schools_complete.csv
schools_csv_path = "raw_data/schools_complete.csv"

# Read the schools CSV into a Pandas DataFrame
schools_df = pd.read_csv(schools_csv_path)

In [ ]:
# Create a reference the CSV students_complete.csv
students_csv_path = "raw_data/students_complete.csv"

# Read the students CSV into a Pandas DataFrame
students_df = pd.read_csv(students_csv_path)
# students_df.columns

In [ ]:
# Check if students does not have missing data
# students_df.count()


In [ ]:
#  Rename column 'name'of schools dataframe and merge both dataframes (tables) 

schools2_df = schools_df.rename(columns={'name': 'school'})
students_schools_df = pd.merge(students_df, schools2_df, on="school")
# students_schools_df.head()


In [ ]:
# PART I.- District Summary
#
# Create a high level snapshot (in table form) of the district's key metrics, including:

# Total Schools
total_schools_d = schools_df['School ID'].count()

# Total Students
total_students_d = students_schools_df['Student ID'].count()

# Total Budget
total_budget_d = schools_df['budget'].sum()

# Average Math Score
average_math_d = students_schools_df['math_score'].mean()

# Average Reading Score
average_read_d = students_schools_df['reading_score'].mean()   

# % Passing Math  & % Passing Reading (passing requires a score of 70% or greater)

# Sort out any Reading Score and Math Score that Fail 
D_passmath_df = students_schools_df.loc[(students_schools_df["math_score"] >= 70)]
D_passread_df = students_schools_df.loc[(students_schools_df["reading_score"] >= 70)]

# % Passing Math
percentage_math_d = ( D_passread_df['Student ID'].count() /
                      students_schools_df['Student ID'].count())*100    

# % Passing Reading
percentage_read_d = ( D_passmath_df['Student ID'].count() /
                      students_schools_df['Student ID'].count())*100  

# Overall Passing Rate (Average of the above two)

pass_rate_d = (percentage_math_d + percentage_read_d)/2

In [ ]:
# Place all of the data  of District Summary found into a summary DataFrame

summary_district_df = pd.DataFrame ({ "Total Schools":['{:,.0f}'.format(total_schools_d)],
                                      "Total Students":['{:,.0f}'.format(total_students_d)],
                                      "Total Budget":['${:,.2f}'.format(total_budget_d)],
                                      "Average Math Score":['{:.2f}'.format(average_math_d)],
                                      "Average Reading Score":['{:.2f}'.format(average_read_d)],
                                      "% Passing Math":['{:.2f}%'.format(percentage_math_d)],
                                      "% Passing Reading":['{:.2f}%'.format(percentage_read_d)],
                                      "Overall Passing Rate":['{:.2f}%'.format(pass_rate_d)]
                                    })

summary_district_df = summary_district_df[["Total Schools","Total Students","Total Budget", "Average Math Score",
                                           "Average Reading Score","% Passing Math", "% Passing Reading",
                                           "Overall Passing Rate"  ]]


summary_district_df

In [ ]:

# PART II.- School Summary

# Create an overview table that summarizes key metrics about each school, including:
# - School Name
# - School Type
# - Total Students
# - Total School Budget
# - Per Student Budget
# - Average Math Score
# - Average Reading Score
# - % Passing Math
# - % Passing Reading
# - Overall Passing Rate (Average of the above)

# Use schools_df_idx dataframe for adding results, School Name is unique value and used as an index.
schools_summary_df = schools_df.set_index('name')

In [ ]:
# Add a column to schools_df_idx_name student_budget 
schools_summary_df['student_budget'] = (schools_summary_df['budget']/schools_summary_df['size'])

# Add a column to schools_df_idx_name average_math for Average Math Score
schools_summary_df['average_math'] = students_schools_df.groupby('school')["math_score"].mean()

# Add a column to schools_df_idx_name average_read for Average Reading Score
schools_summary_df['average_reading'] = students_schools_df.groupby('school')["reading_score"].mean()

# % Passing Math  & % Passing Reading (passing requires a score of 70% or greater)

# Leave out the scores are below 70% and group by school_id
reduced_math_school_df = students_schools_df.loc[(students_schools_df["math_score"] >= 70)]

reduced_read_school_df = students_schools_df.loc[(students_schools_df["reading_score"] >= 70)]

# Total_math_school and total_read_school are lists
total_math_school = reduced_math_school_df.groupby('School ID')['Student ID'].count()
total_read_school = reduced_read_school_df.groupby('School ID')['Student ID'].count()

# Add columns with total_math_school & total_read_school from respectively series
schools_summary_df['total_math_school'] = pd.Series(total_math_school).values
schools_summary_df['total_read_school'] = pd.Series(total_read_school).values

# Calculate and Add columns perc_math_school and perc_read_school
schools_summary_df['perc_math_school'] = (  pd.Series(total_math_school).values /
                                            schools_summary_df['size'])*100

schools_summary_df['perc_read_school'] = (  pd.Series(total_read_school).values /
                                            schools_summary_df['size'])*100

# Add Overall Passing Rate
schools_summary_df['over_pass_rate'] = ( schools_summary_df['perc_math_school'] + 
                                         schools_summary_df['perc_read_school'])/2

In [ ]:
# Reorganizing the columns using double brackets, type is School Type, size is Total Students, 
# and budget is Total School Budget.

# Using .rename(columns={}) in order to rename columns for desire output
school_summary_df = schools_summary_df.rename( columns=
                                             {  "type":"School Type", "size":"Total Students", 
                                                "budget":"Total School Budget", "student_budget": "Per Student Budget",
                                                "average_math": "Average Math Score", "average_reading": "Average Reading Score",
                                                "perc_math_school": "% Passing Math", "perc_read_school": "% Passing Reading",
                                                "over_pass_rate": "% Overall Passing Rate"
                                             })

output_schools_df = school_summary_df.loc[: ,[ "School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                               "Average Math Score", "Average Reading Score","% Passing Math", 
                                               "% Passing Reading","% Overall Passing Rate"]]

# Formatting or mapping
output_schools_df['Total Students'] = output_schools_df['Total Students'].map('{:,.0f}'.format)
output_schools_df['Total School Budget'] = output_schools_df['Total School Budget'].map('${:,.2f}'.format)
output_schools_df['Per Student Budget'] = output_schools_df['Per Student Budget'].map('${:,.2f}'.format)
output_schools_df['Average Math Score'] = output_schools_df['Average Math Score'].map('{:.2f}'.format)
output_schools_df['Average Reading Score'] = output_schools_df['Average Reading Score'].map('{:.2f}'.format)
output_schools_df['% Passing Math'] = output_schools_df['% Passing Math'].map('{:.2f}%'.format)
output_schools_df['% Passing Reading'] = output_schools_df['% Passing Reading'].map('{:.2f}%'.format)
output_schools_df['% Overall Passing Rate'] = output_schools_df['% Overall Passing Rate'].map('{:.2f}%'.format)

output_schools_df 

In [ ]:

# PART III .- Top Performing Schools (By Passing Rate)

# Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
output_schools_df.sort_values(by="% Overall Passing Rate", ascending=False).head()

In [ ]:

# PART IV .- Bottom Performing Schools (By Passing Rate)

# Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:

output_schools_df.sort_values(by="% Overall Passing Rate").head()


In [ ]:
# PART V .- Math Scores by Grade 

# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# Using merged_df which joins students_complete.csv and schools_complete.csv

students_schools_df_idx = students_schools_df.set_index(['School ID','Student ID'])

# Get 9th, 10th, 11th and 12th grades and group by school id

grade_9th_df = students_schools_df.loc[students_schools_df["grade"] == "9th", :]
grade_10th_df = students_schools_df.loc[students_schools_df["grade"] == "10th", :]
grade_11th_df = students_schools_df.loc[students_schools_df["grade"] == "11th", :]
grade_12th_df = students_schools_df.loc[students_schools_df["grade"] == "12th", :]

# Group by school id

math_9th_df = grade_9th_df.groupby(['school']).mean()['math_score']
math_10th_df = grade_10th_df.groupby(['school']).mean()['math_score']
math_11th_df = grade_11th_df.groupby(['school']).mean()['math_score']
math_12th_df = grade_12th_df.groupby(['school']).mean()['math_score']

# Create a Dataframe that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

MathScoresByGrade_df = pd.DataFrame({ "9th" : math_9th_df, "10th" : math_10th_df, 
                                     "11th" : math_11th_df, "12th": math_12th_df
                                    })

MathScoresByGrade_df = MathScoresByGrade_df [[ "9th", "10th", "11th", "12th"]]

MathScoresByGrade_df.head(14)


In [ ]:
# PART VI .- Reading Scores by Grade

# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

reading_9th_df = grade_9th_df.groupby(['school']).mean()['reading_score']
reading_10th_df = grade_10th_df.groupby(['school']).mean()['reading_score']
reading_11th_df = grade_11th_df.groupby(['school']).mean()['reading_score']
reading_12th_df = grade_12th_df.groupby(['school']).mean()['reading_score']

ReadingScoresByGrade_df = pd.DataFrame({ "9th" : reading_9th_df, "10th" : reading_10th_df, 
                                        "11th" : reading_11th_df, "12th": reading_12th_df
                                    })

ReadingScoresByGrade_df = ReadingScoresByGrade_df [[ "9th", "10th", "11th", "12th"]]

ReadingScoresByGrade_df

In [ ]:
# PART VII .- Scores by School Spending

# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
# Use 4 reasonable bins to group school spending. Include in the table each of the following:

# 1. Average Math Score
# 2. Average Reading Score
# 3. % Passing Math
# 4. % Passing Reading
# 5. Overall Passing Rate (Average of the above two)

per_student_budget_df=school_summary_df.sort_values(by="Per Student Budget")


In [ ]:
# Create bins in which to place values based upon Per Student Budget
budget_bins = [0, 585, 615, 645, 675]

# Create labels for these bins
budget_labels = ["<$585", "$585-615", "$615-645", "$645-675"]

# Slice the data and place the data series into a new column inside of the DataFrame
per_student_budget_df["Spending Ranges (Per Student)"] = pd.cut(per_student_budget_df["Per Student Budget"],
                                                      budget_bins,labels=budget_labels)

math_scores = per_student_budget_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
reading_scores = per_student_budget_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
passing_math = per_student_budget_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
passing_reading = per_student_budget_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
passing_rate = (passing_math + passing_reading) / 2

# Create  a data frame to list the Performance based on School Spending

school_spending_summary_df = pd.DataFrame({"Average Math Score" : math_scores, "Average Reading Score": reading_scores,
                                           "% Passing Math": passing_math, "% Passing Reading": passing_reading,
                                           "% Overall Passing Rate": passing_rate})

school_spending_summary_df = school_spending_summary_df[["Average Math Score", "Average Reading Score", 
                                                        "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

school_spending_summary_df


In [ ]:
# PART VIII .- Scores by School Size

# Repeat the above breakdown, but this time group schools based on a reasonable approximation 
# of school size (Small, Medium, Large).

per_school_size_df=school_summary_df.sort_values(by="Total Students")


In [ ]:
# Create bins in which to place values based upon School Size
school_bins = [0, 1000, 2000, 5000]

# Create labels for these bins
school_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Slice the data and place the data series into a new column inside of the DataFrame
per_school_size_df["School Size"] = pd.cut(per_school_size_df["Total Students"], school_bins,labels=school_labels)

# Do calculations
math_scores_school = per_school_size_df.groupby(["School Size"]).mean()["Average Math Score"]
reading_scores_school = per_school_size_df.groupby(["School Size"]).mean()["Average Reading Score"]
passing_math_school = per_school_size_df.groupby(["School Size"]).mean()["% Passing Math"]
passing_reading_school = per_school_size_df.groupby(["School Size"]).mean()["% Passing Reading"]
passing_rate_school = (passing_math_school + passing_reading_school) / 2

# Create  a data frame to list performance based on School Size

school_size_summary_df = pd.DataFrame({"Average Math Score" : math_scores_school, "Average Reading Score": reading_scores_school,
                                           "% Passing Math": passing_math_school, "% Passing Reading": passing_reading_school,
                                           "% Overall Passing Rate": passing_rate_school})

school_size_summary_df = school_size_summary_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                                 "% Passing Reading", "% Overall Passing Rate"]]

school_size_summary_df

In [ ]:
# PART IX .- Scores by School Type

# Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).


In [ ]:
grouped_school_type_df = school_summary_df.groupby("School Type")

math_scores_type = grouped_school_type_df["Average Math Score"].mean()
reading_scores_type = grouped_school_type_df["Average Reading Score"].mean()
passing_math_type = grouped_school_type_df["% Passing Math"].mean()
passing_reading_type = grouped_school_type_df["% Passing Reading"].mean()
passing_rate_type = (passing_math_type + passing_reading_type) / 2

# Create  a data frame to list performance based on School Type

school_type_summary_df = pd.DataFrame({"Average Math Score" : math_scores_type, "Average Reading Score": reading_scores_type,
                                       "% Passing Math": passing_math_type, "% Passing Reading": passing_reading_type,
                                       "% Overall Passing Rate": passing_rate_type})

school_type_summary_df = school_type_summary_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                                 "% Passing Reading", "% Overall Passing Rate"]]
school_type_summary_df